<a href="https://colab.research.google.com/github/ncn-foreigners/job-ads-classifier/blob/main/job_classification_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab and environment settings

Printing in colab

In [1]:
from google.colab.data_table import DataTable
DataTable.max_columns = 60

Clone repository

In [2]:
!git clone https://github.com/ncn-foreigners/job-ads-classifier.git 

Cloning into 'job-ads-classifier'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 127 (delta 48), reused 52 (delta 8), pack-reused 0
Receiving objects: 100% (127/127), 2.18 MiB | 3.81 MiB/s, done.
Resolving deltas: 100% (48/48), done.


Set-up virtual env

In [3]:
!pip install virtualenv
!virtualenv classifier
!source /content/classifier/bin/activate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 29.7 MB/s 
     |████████████████████████████████| 468 kB 68.1 MB/s 
created virtual environment CPython3.7.15.final.0-64 in 800ms
  creator CPython3Posix(dest=/content/classifier, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.2.2, setuptools==65.3.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


Upgrade pip

In [4]:
!python -m pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pip-22.3-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


Download models

In [5]:
!gdown 1N2LAJyG5tZT5UjHWRUFOgia5dSPwz1eg
!unzip models-v1.0.zip
!rm -rf __MACOSX
!mv models ./job-ads-classifier/

Downloading...
From: https://drive.google.com/uc?id=1N2LAJyG5tZT5UjHWRUFOgia5dSPwz1eg
To: /content/models-v1.0.zip
100% 1.87G/1.87G [00:40<00:00, 46.0MB/s]
Archive:  models-v1.0.zip
   creating: models/
  inflating: models/.DS_Store        
  inflating: __MACOSX/models/._.DS_Store  
   creating: models/linear_model/
  inflating: __MACOSX/models/._linear_model  
   creating: models/transformer_model/
  inflating: __MACOSX/models/._transformer_model  
  inflating: models/linear_model/X_train.bin  
  inflating: __MACOSX/models/linear_model/._X_train.bin  
  inflating: models/linear_model/napkinxc_hierarchy.bin  
  inflating: __MACOSX/models/linear_model/._napkinxc_hierarchy.bin  
  inflating: models/linear_model/tfidf_vectorizer.bin  
  inflating: __MACOSX/models/linear_model/._tfidf_vectorizer.bin  
  inflating: models/linear_model/weights.bin  
  inflating: __MACOSX/models/linear_model/._weights.bin  
  inflating: models/linear_model/Y_train.bin  
  inflating: __MACOSX/models/linear_mod

Install required packages

In [6]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install -r /content/job-ads-classifier/requirements-py37-colab.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 409.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.5/541.5 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 89.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.7/361.7 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.1/582.1 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 52.9 MB/s eta 0:00:00


In [8]:
exit()

## Working with trained models

In [8]:
models_dir="/content/job-ads-classifier/models"

Load required modules

In [3]:
import sys
sys.path.insert(1, '/content/job-ads-classifier')

In [4]:
import numpy as np
import pandas as pd
from job_offers_classifier.load_save import load_to_df, save_obj, load_obj
from job_offers_classifier.job_offers_classfier import *
import torch
from torchmetrics import MetricCollection, Recall, Accuracy, ConfusionMatrix
execfile("/content/job-ads-classifier/functions.py")

Example data -- job offers from Public Sercice

In [26]:
#kprm_df = pd.read_xml("https://nabory.kprm.gov.pl/pls/serwis/app.xml", xpath = "./new/oferta")
kprm_df = pd.read_xml("app.xml", xpath = "./new/oferta")
kprm_df = kprm_df.replace("\\<\\!\\[CDATA\\[ ", "", regex=True)
kprm_df = kprm_df.replace(" \\]\\]\\>", "", regex=True)
kprm_df = kprm_df.replace(r'<[^<>]*>', ' ', regex=True)
kprm_df = kprm_df.replace('&nbsp;', ' ', regex=True)
kprm_df = kprm_df.replace('&dash;', ' ', regex=True)
kprm_df = kprm_df.replace('&oacute;', ' ', regex=True)
kprm_df = kprm_df.replace('\r', ' ', regex=True)
kprm_df['desc'] = kprm_df[['stanowisko', 'do_spraw', 'nazwa_firmy', 'poledodatkowelista1', 'poledodatkowelista3']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
kprm_df[["unid", "nazwa_firmy", "stanowisko", "do_spraw", "poledodatkowelista1", "poledodatkowelista3"]].head(n=2)

,unid,nazwa_firmy,stanowisko,do_spraw,poledodatkowelista1,poledodatkowelista3
0,108789,Wojewódzki Urząd Ochrony Zabytków w Warszawie,inspektor ochrony zabytków,None,Prowadzenie postępowań w sprawach - prac kon...,"praca biurowa, praca w siedzibie wydzia..."
1,108788,Wojewódzki Inspektorat Nadzoru Budowlanego w W...,młodszy specjalista,orzecznictwa administracyjnego,analizowanie dokumentacji budowlanej dla pot...,Zadania wykonywane na stanowisku wiążą się z:...


In [27]:
kprm_df.desc[0]

'inspektor ochrony zabytków None Wojewódzki Urząd Ochrony Zabytków w Warszawie   Prowadzenie postępowań w sprawach - prac konserwatorskich, restauratorskich, robót budowlanych, oraz badań przy obiektach zabytkowych, zespołach stanowiących zabytki i obszarach objętych ochroną konserwatorską - przygotowanie projektów postanowień, decyzji, opinii i zaleceń konserwatorskich.  Prowadzenie postępowań, przygotowywanie opinii i uzgodnień wynikających z problematyki ustawy o zagospodarowaniu przestrzennym i ustawy Prawo budowlane;  Prowadzenie inspekcji obiektów, w których prowadzone są prace konserwatorskie i rewaloryzacyjne, w celu stwierdzenia zgodności tych prac z zatwierdzoną dokumentacją i wydanym pozwoleniem oraz potwierdzenie zakresu wykonanych prac w stosunku do wysokości dofinansowania w przypadku obiektów objętych dofinansowaniem i sporządzenie protokołów odbioru;  Prowadzenie terenowych inspekcji zabytków w celu sprawdzenia stanu ich zachowania, sposobu użytkowania, stanu prawno-wła

### Logistic regression model using tf-df matrix

In [9]:
model_paper_linear = LinearJobOffersClassifier()
model_paper_linear.load(f"{models_dir}/linear_model")

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [28]:
kprm_df_zaw1 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

Processing X ...
Processing text ...



100%|██████████| 450/450 [00:01<00:00, 239.94it/s]


Transforming text to tf-idf ...
Predicting ...



100%|██████████| 450/450 [00:00<00:00, 1818.93it/s]


Distribution of occupancy codes

In [29]:
np.sum(kprm_df_zaw1, axis=0)

0      0.030328
1     18.140692
2    215.555435
3    176.278656
4     30.673830
5      4.925883
6      0.543263
7      1.986461
8      1.147614
9      0.697216
dtype: float32

Top 5 occupancy codes

In [30]:
kprm_df_zaw6_top5 = model_paper_linear.predict(
    X_text=list(kprm_df["desc"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Processing X ...
Processing text ...



100%|██████████| 450/450 [00:01<00:00, 241.84it/s]


Transforming text to tf-idf ...
Predicting ...



100%|██████████| 450/450 [00:00<00:00, 1800.54it/s]


In [31]:
kprm_df_zaw6_top5.head(n=5)

,class_1,class_2,class_3,class_4,class_5,prob_1,prob_2,prob_3,prob_4,prob_5
0,265107,242217,242211,242290,265105,0.608035,0.110593,0.064497,0.015760,0.013228
1,242211,261901,261103,261990,262208,0.669907,0.100426,0.046531,0.024558,0.015696
2,242217,213303,242290,242232,213390,0.343668,0.277495,0.030850,0.027814,0.027478
3,441590,242307,121201,431301,242310,0.360840,0.281625,0.188249,0.058488,0.006177
4,325512,225211,324002,225290,225212,0.751640,0.051141,0.034717,0.030207,0.020690


### Transformer model

In [32]:
model_paper_trans = TransformerJobOffersClassifier(gpus=1, threads=2)
model_paper_trans.load(f"{models_dir}/transformer_model")

Initializing TransformerClassifier with model_name=allegro/herbert-base-cased, output_type=linear, num_labels=2905, learning_rate=1e-05, weight_decay=0.01, warmup_steps=50 ...


Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
kprm_df_zaw1_trans = model_paper_trans.predict(
    X=list(kprm_df["desc"]),
    output_level=0,
    format="dataframe"
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

Distribution of occupancy codes

In [34]:
np.sum(kprm_df_zaw1_trans, axis=0)

0      0.078096
1     25.473125
2    179.177933
3    135.865738
4     43.363914
5     12.509176
6      3.247363
7     24.571152
8     18.559874
9      7.153627
dtype: float32

In [35]:
kprm_df_zaw6_top5_trans = model_paper_trans.predict(
    X=list(kprm_df["desc"]),
    output_level="last",
    format="dataframe",
    top_k=5
)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

In [36]:
kprm_df_zaw6_top5_trans.head(n=5)

,class_1,class_2,class_3,class_4,class_5,prob_1,prob_2,prob_3,prob_4,prob_5
0,265107,242217,711103,242211,411003,0.879289,0.020518,0.006282,0.003679,0.000992
1,242211,242217,411003,411090,261103,0.997107,0.000516,0.000031,0.000027,0.000017
2,242217,325504,213303,242204,411003,0.703757,0.046418,0.012640,0.003954,0.003184
3,242307,121201,441590,411003,431190,0.646910,0.158532,0.111106,0.013167,0.002904
4,325512,225212,324002,225101,225211,0.850876,0.025001,0.020727,0.005644,0.004081


Compare linear vs transformer

In [37]:
kprm_df["lin"] = kprm_df_zaw6_top5["class_1"]
kprm_df["trans"] = kprm_df_zaw6_top5_trans["class_1"]
kprm_df[kprm_df["lin"].str.slice(0,1) != kprm_df["trans"].str.slice(0,1)][
    ["trans", "lin", "stanowisko", "do_spraw", "desc"]].head(n=5)

,trans,lin,stanowisko,do_spraw,desc
3,242307,441590,referent,ds. kadr i płac,referent ds. kadr i płac Powiatowy Inspektorat...
12,242204,132102,audytor wewnętrzny,None,audytor wewnętrzny None Ministerstwo Spraw Zag...
18,411003,341103,podreferendarz,obsługi kancelaryjnej,podreferendarz obsługi kancelaryjnej Ministers...
21,311104,216590,inspektor wojewódzki,państwowego zasobu geodezyjnego i kartograficz...,inspektor wojewódzki państwowego zasobu geodez...
25,411003,242217,referent,sekretariatu,referent sekretariatu Izba Administracji Skarb...


Looking at the predictions and descriptions it seems that: TO 
UPDATE

## Compare with official dictionary 

Selected occupancies from official dictionary to see which model is better.

1. 712401 -- monter izolacji budowlanych
2. 214102 -- inżynier organizacji i planowania produkcji
3. 932101 -- pakowacz ręczny


In [38]:
opisy = ["Monter dociepleń budynków - Francja, okolice Grenoble , Zagranica od ATERIMA WORK FR Monter dociepleń budynków Francja, okolice Grenoble, Zagranica Umowa o pracę tymczasową Obowiązki:montaż zewnętrznej izolacji termicznej (styropian) malowanie elewacji montaż / demontaż rusztowań niezbędnych do wykonania pracy Wymagania:doświadczenie na podobnym stanowisku dyspozycyjność do podjęcia zatrudnienia od zaraz prawo jazdy kat. B oraz mile widziany własny samochód do dyspozycji badania wysokościowe na podstawie naszego skierowania Wybranym kandydatom oferujemy:legalne zatrudnienie na terenie Francji u sprawdzonego klienta umowę o pracę tymczasową do 30 lipca z możliwością przedłużenia 12,90 EUR brutto / h (za 35 godzin tygodniowo) dodatek wyżywieniowy 10,30 EUR brutto za każdy dzień roboczy zapewnione nadgodziny płatne dodatkowo (powyżej 35 godz.: 16,12 EUR brutto/h, powyżej 43 godz.: 19,35 EUR brutto/h) zawsze terminową wypłatę wynagrodzenia składki ZUS opłacane od średniej krajowej bezpłatne zakwaterowanie częściowy zwrot kosztów podróży do Francji Co jest dla Ciebie ważne? Wiemy, że do dobrej pracy potrzebne są odpowiednie warunki. Dobrze znamy naszych kandydatów i świetnie rozumiemy, na czym szczególnie im zależy w pracy. Obowiązki:montaż zewnętrznej izolacji termicznej (styropian) malowanie elewacji montaż / demontaż rusztowań niezbędnych do wykonania pracy Wymagania:doświadczenie na podobnym stanowisku dyspozycyjność do podjęcia zatrudnienia od zaraz prawo jazdy kat. B oraz mile widziany własny samochód do dyspozycji badania wysokościowe na podstawie naszego skierowania Wybranym kandydatom oferujemy:legalne zatrudnienie na terenie Francji u sprawdzonego klienta umowę o pracę tymczasową do 30 lipca z możliwością przedłużenia 12,90 EUR brutto / h (za 35 godzin tygodniowo) dodatek wyżywieniowy 10,30 EUR brutto za każdy dzień roboczy zapewnione nadgodziny płatne dodatkowo (powyżej 35 godz.: 16,12 EUR brutto/h, powyżej 43 godz.: 19,35 EUR brutto/h) zawsze terminową wypłatę wynagrodzenia składki ZUS opłacane od średniej krajowej bezpłatne zakwaterowanie częściowy zwrot kosztów podróży do Francji",
         "Specjalista ds. planowania Twój zakres obowiązkówplanowanie produkcji zgodnie z polityką i procedurami firmyoptymalizacja i budowa efektywnych modeli planistycznych (proces ciągły)wywoływanie zakupów i rezerwacja surowców do zleceń produkcyjnychanaliza rotacji i optymalizacja poziomu zapasu surowców (proces ciągły)udział w projektach wewnętrznych optymalizujących procesy planowania i zakupówraportowanie wyników KPI w podległym obszarze Nasze wymaganiaminimum dwuletnie doświadczenie w planowaniu produkcji z wykorzystaniem systemu klasy ERPdoświadczenie w obszarze zakupów surowców i materiałów do produkcjiwykształcenie wyższego (preferowane kierunki ekonomia, inżynieria produkcji, logistyka)umiejętność analizy danych, wyciągania wniosków oraz proponowania rozwiązańznajomość języka angielskiego w stopniu komunikatywnym (B2)znajomość obsługi komputera i pakietu MS Office - w szczególności Excel na poziomie minimum średniozaawansowanym – warunek koniecznysamodzielność w działaniu i umiejętność podejmowania decyzjidobra organizacji pracy i zarządzania czasem oraz umiejętności ustalania priorytetów – warunek koniecznyMile widzianedoświadczenie we wdrażaniu systemów do planowania klasy ERPdoświadczenie w diagnozie i modelowaniu procesów produkcyjnych To oferujemystabilne zatrudnienie w oparciu o umowę o pracę w pełnym wymiarze godzinpracę w zespole nastawionym na współpracę, dzielenie się wiedzą i wspólny szacunekatrakcyjne wynagrodzenie aby docenić Twoje kompetencjepełną wyzwań pracę w dziale Zarządzania Łańcuchem Dostaw w firmie produkcyjnejmożliwość rozwoju i codziennego doskonalenia Twoich kwalifikacji zawodowychudział w projektach wewnętrznych optymalizujących nasze procesypracę od poniedziałku do piątku w godzinach od 08:00 do 16:00pakiet benefitów pozapłacowych w tym ubezpieczenie na życiepakiet socjalny w ramach ZFŚS",
         "Pakowacz Zakres obowiązków:- pakowanie towaru- etykietowanie towaru- ważenie towaruWymagania:Wymagania konieczne: Wykształcenie: podstawowe, brakWynagrodzenie brutto: od 2 800 PLNRodzaj umowy:Umowa o pracę na czas określonyLiczba wolnych miejsc pracy:10, dla niepełnosprawnych 1Sposób aplikowania:bezpośrednio do pracodawcyPracodawca:Zakład przemysłu mięsnego biernacki sp. z o.o.Profil działalności pracodawcy:przetwarzanie i konserwowanie mięsa,z wyłączeniem mięsa z drobiuMiejsce pracy: Dworcowa 47 D, 63-200 Golina, powiat: jarociński, woj: wielkopolskieKontakt:Preferowane formy kontaktu: telefon, e-mailNumer telefonu:62 747 09 27Wymagane dokumenty:CVData rozpoczęcia pracy:13.07.2021Data ważności ogłoszenia:30.09.2021Dodane przez:Powiatowy Urząd Pracy w JarocinieIdentyfikator oferty:StPr/21/0506"]

In [39]:
trans3=kprm_df_zaw1_trans = model_paper_trans.predict(X=list(opisy), output_level="last", format="dataframe", top_k=1)
lin3=kprm_df_zaw1_trans = model_paper_linear.predict(X_text=list(opisy), output_level="last", format="dataframe", top_k=1)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  category=PossibleUserWarning,
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

Processing X ...
Processing text ...



100%|██████████| 3/3 [00:00<00:00, 164.05it/s]


Transforming text to tf-idf ...
Predicting ...



100%|██████████| 3/3 [00:00<00:00, 1732.94it/s]


In [40]:
results = pd.concat([trans3,lin3], axis = 1)
results["correct"] = pd.Series(["712401", "214102", "932101"])
results

,class_1,prob_1,class_1,prob_1,correct
0,712301,0.528665,712401,0.843104,712401
1,214102,0.706504,214102,0.840713,214102
2,932101,0.990134,932101,0.646111,932101


## Evaluation on data sources 

In [18]:
ksid_dict = pd.read_csv("desc_praca_gov_pl_2022_03_30.csv", dtype='str')
ksid_dict = ksid_dict[ksid_dict["class"].isin(list(model_paper_linear.hierarchy.keys()))]
#ksid_dict = ksid_dict[ksid_dict["occup_code"].str.slice(0,1) == "3"]
#ksid_dict.shape
#ksid_dict = pd.read_csv("https://raw.githubusercontent.com/ncn-foreigners/job-ads-datasets/main/data/gus-synonimy.csv", dtype="str")
#ksid_dict2=ksid_dict[(ksid_dict["kod_zawodu1"] == ksid_dict["kod_zawodu2"]) & 
#                     (ksid_dict["kod_zawodu2"] == ksid_dict["kod_zawodu3"]) & 
#                     (ksid_dict["kod_zawodu1"] == ksid_dict["kod_zawodu3"])]
#ksid_dict2 = ksid_dict2[ksid_dict2["kod_zawodu1"].isin(list(model_paper_linear.hierarchy.keys()))]
#ksid_dict2["desc"] = ksid_dict2["job_title"] + " " + ksid_dict2["job_description"]
ksid_dict.head(n=1)

,class,desc
0,011001,Oficer zawodowy Sił Zbrojnych Oficer zawodowy ...


In [19]:
last_level_pred_lin, last_level_map_lin = model_paper_linear.predict(
    X_text=list(ksid_dict["desc"]),
    X_matrix=None,
    output_level='last',
    top_k=None,
    format="array"
)
model_paper_linear_results = evaluate_levels(model_paper_linear, 
                                             ksid_dict["class"], 
                                             last_level_pred_lin, 
                                             last_level_map_lin, 
                                             levels=[0, 1, 2, 3, 4])
pd.DataFrame(model_paper_linear_results)

Processing X ...
Processing text ...


100%|██████████| 2903/2903 [00:10<00:00, 275.44it/s]


Transforming text to tf-idf ...
Predicting ...


100%|██████████| 2903/2903 [00:01<00:00, 1987.32it/s]


,level 0,level 1,level 2,level 3,level 4
acc/recall@1,0.960730,0.957286,9.565966e-01,9.552187e-01,9.593524e-01
macro_acc,0.960820,0.950556,9.518046e-01,9.454335e-01,9.593524e-01
recall@2,0.994488,0.993800,9.882880e-01,9.872546e-01,9.844988e-01
recall@3,0.999311,0.997589,9.944885e-01,9.927661e-01,9.903548e-01
recall@4,0.999656,0.998967,9.958664e-01,9.944885e-01,9.920772e-01
recall@5,1.000000,0.999656,9.968998e-01,9.951774e-01,9.934551e-01
recall@6,1.000000,0.999656,9.975887e-01,9.962108e-01,9.948329e-01
recall@7,1.000000,0.999656,9.986221e-01,9.972442e-01,9.962108e-01
recall@8,1.000000,1.000000,9.996555e-01,9.979331e-01,9.965553e-01
recall@9,1.000000,1.000000,9.996555e-01,9.986221e-01,9.975887e-01


In [24]:
last_level_pred_tra, last_level_map_tra = model_paper_trans.predict(
    X=list(ksid_dict["desc"]),
    output_level='last',
    top_k=None,
    format="array"
)
model_paper_trans_results = evaluate_levels(model_paper_trans, 
                                             ksid_dict["class"], 
                                              last_level_pred_tra, 
                                             last_level_map_tra, 
                                             levels=[0, 1, 2, 3, 4])
pd.DataFrame(model_paper_trans_results)

Initializing TransformerDataModule with model_name=allegro/herbert-base-cased, max_seq_length=512, train/eval_batch_size=8/8, num_workers=2 ...
Setting up TransformerDataModule ...


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Starting predicting with TransformerClassifier ...


INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.utilities.rank_zero:Loaded model weights from checkpoint at /content/job-ads-classifier/models/transformer_model/transformer_classifier.ckpt


Predicting: 0it [00:00, ?it/s]

,level 0,level 1,level 2,level 3,level 4
acc/recall@1,0.902170,0.915605,9.421288e-01,9.204271e-01,9.459180e-01
macro_acc,0.801080,0.885401,9.260226e-01,9.240519e-01,9.459180e-01
recall@2,0.956252,0.942129,9.683086e-01,9.720978e-01,9.838098e-01
recall@3,0.982432,0.956252,9.758870e-01,9.882880e-01,9.931106e-01
recall@4,0.986910,0.964864,9.803652e-01,9.913882e-01,9.972442e-01
recall@5,0.995522,0.973131,9.851877e-01,9.934551e-01,9.979331e-01
recall@6,0.999311,0.986566,9.913882e-01,9.962108e-01,9.979331e-01
recall@7,0.999656,0.994144,9.934551e-01,9.972442e-01,9.986221e-01
recall@8,1.000000,0.997244,9.962108e-01,9.986221e-01,9.989666e-01
recall@9,1.000000,0.998278,9.975887e-01,9.989666e-01,9.989666e-01
